In [2]:
## Side packages
from numba import double 
from numba.decorators import jit, autojit
import math
import numpy as np
import netCDF4 as nc
from os import listdir
from os.path import isfile, join
import datetime
import multiprocessing
import pandas as pd

In [ ]:
# load Drone spec table
df = pd.read_excel(r'C:\Users\mozhou.gao\Desktop\2018_07_24\NewDroneTable.xlsx')

# examine table 
print (df.head())

# create drone types index 
drone_index = np.arange(0, 12, 1)

In [ ]:
# Read weather data
mypath1 = r'Y:\Mo_GlobalUAVMapping\High_res_data\Wind_Temp'
Windfiles = [f for f in listdir(mypath1) if isfile(join(mypath1, f))]
mypath2 = r'Y:\Mo_GlobalUAVMapping\High_res_data\Precip'
Precipfiles = [f for f in listdir(mypath2) if isfile(join(mypath2, f))]

# create weather data name list  
name = [2007,2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]
files = zip(name,Windfiles,Precipfiles)
files = list(files)
print(files)

In [ ]:
def UAV_Mapping(files,W,P,T,D_type,DL):
    
    '''
    files:weatherfile 
    W: maximum wind resistance
    P: Precipitation threshold 
    T: Operating temperature range 
    D_type: Drone model name 
    DL: whether to turn on the daylight calculation 
    '''
    
    # for each year of weather data 
    for n in files:
        print ("year: " + str (n[0]))
        T1 = datetime.datetime.now()
        pfile = nc.Dataset(r"Y:\Mo_GlobalUAVMapping\High_res_data\Precip\%s"%n[2], 'r')
        wtfile = nc.Dataset(r"Y:\Mo_GlobalUAVMapping\High_res_data\Wind_Temp\%s"%n[1], 'r')
        # extract latitude, longitude, and time
        time = pfile.variables['time'][:]
        lat = pfile.variables['latitude'][:]
        lon = pfile.variables['longitude'][:]
        # Check the Dimension
        t = len(time)
        la = len(lat)
        lo = len(lon)
        if DL == True: 
            print ('Loading Daylight...')
            # Check leap Year
            if t == 2928:
                y = 366
                dayfile = nc.Dataset(r"Y:\Mo_GlobalUAVMapping\High_res_data\leapyear.nc","r")
            else:
                y = 365
                dayfile = nc.Dataset(r"Y:\Mo_GlobalUAVMapping\High_res_data\normalyear.nc","r")

        else: 
            print ('shutting down Daylight...')
        index = np.arange(t)
        # create empty data matrix for storing dronedays 
        data = np.empty((len(time),len(lat),len(lon)),dtype = np.float)
        for i in index:
            u = wtfile.variables['u10'][i, :, :]
            v = wtfile.variables['v10'][i, :, :]
            temp = wtfile.variables['t2m'][i, :, :]
            prep = pfile.variables['tp'][i, :, :]
            day = dayfile.variables['daytime'][i,:,:]
            # PreProcessing
            wind = (u ** 2 + v ** 2) ** 0.5
            temp = temp - 273.15
            # Array Filter
            # Wind
            wa = wind <= W
            wb = wind > W
            wind[wa] = 1
            wind[wb] = 0

            # Precipitation
            pa = prep < P
            pb = prep >= P
            prep[pa] = 1
            prep[pb] = 0

            # Temperature
            ta = (temp <= T[1]) & (temp >= T[0])
            tb = (temp > T[1]) | (temp < T[0])
            temp[ta] = 1
            temp[tb] = 0
            
            # DroneDay
            Overall = wind + temp + prep + day
            oa = Overall == 4
            ob = Overall != 4
            Overall[oa] = 1
            Overall[ob] = 0
            data[i, :, :] = Overall
            
            print (i)

        # Covert Drone 3hour to Dronday

        ngrid = np.zeros(shape=(y,la,lo))
        index = np.linspace(8, t, y)
        index = index.astype(int)

        k = 0
        for ti in index:

            ui = ti - 1
            li = ti - 8
            cell = data[li:ui,:,:]
            s = np.sum(cell, axis=0)
            s[s > 0] = 1
            ngrid[k, :, :] = s
            print (k)
            k = k + 1
         
        dday = np.sum(ngrid,axis= 0)
        
        # save dronedays for each drone model
        dronedata =nc.Dataset(r"Y:\Mo_GlobalUAVMapping\High_res_data\NewResult_daylight\Phantom%s" % D_type+str(n[0]), 'w', format='NETCDF4_CLASSIC')
        lat = dronedata.createDimension('lat', la)
        lon = dronedata.createDimension('lon', lo)
        dt = dronedata.createVariable('daytime', np.int32, ('lat', 'lon'))
        dt[:] = dday
    
        # Close netCDF File
        dronedata.close()
        pfile.close()
        wtfile.close()
        dayfile.close()
        T2 = datetime.datetime.now()
        
        #print calculation time 
        print (T2-T1)

In [ ]:
uav_numba = autojit(UAV_Mapping)

In [ ]:
for ind in drone_index:
    T = []
    T.append(df.iloc[ind, :]["TempL"])
    T.append(df.iloc[ind, :]["TempU"])
    W = df.iloc[ind, :]["Wind "]
    P = df.iloc[ind, :]["Precip"]
    typ = df.iloc[ind, :]["Drones"]
    uav_numba(files,W,P,T,typ)